In [ ]:
from mlflow import log_metric, log_param, log_artifacts
import mlflow
import numpy as np
import pandas as pd

import finstar.evaluation.returns as returns
import finstar.strategies.sma as sma

In [ ]:
exp_name = "20210623-sma-SP500-10yrs"
try:
    exp_id = mlflow.create_experiment(exp_name)
except mlflow.exceptions.MlflowException:
    exp_id = mlflow.get_experiment_by_name(exp_name).experiment_id

In [ ]:
df = pd.read_parquet("../data/interim/20210622-SP500-10yrs/20210622-SP500-10yrs.snappy.parquet")

# remove columns with missing data
start_cols = df.shape[1]
df = df.loc[:, df.isna().sum() == 0]
end_cols = df.shape[1]
print(f"Before {start_cols}, then {end_cols} columns.")

with mlflow.start_run(experiment_id=exp_id):
    short_sma = 3
    long_sma = 200
    log_param("short_sma", short_sma)
    log_param("long_sma", long_sma)

    def get_strategy_returns(close_prices):
        pos = sma.sma(close_prices, short_sma, long_sma)
        ret = returns.log_returns(close_prices)
        strat = returns.strategy_returns(pos, ret)
        return np.exp(strat.sum())

    strat_returns = df.apply(get_strategy_returns, axis=0)
    mean_returns = strat_returns.mean()
    log_metric("mean_returns", mean_returns)